In [19]:
!pip install torch
!pip install torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F

import numpy as np

In [29]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=32, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=32, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=32)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [30]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


In [31]:
def train_part(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [32]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [33]:
# Here you should use nn.Sequential to define and train a three-layer ConvNet with the same architecture we used in Part III:

# Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
# ReLU
# Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
# ReLU
# Fully-connected layer (with bias) to compute scores for 10 classes

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

hidden_layer_size = 4000
learning_rate = 1e-2

channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = None
optimizer = None

model = nn.Sequential( nn.Conv2d(3, channel_1, 5, padding = 2),
                       nn.ReLU(),
                       nn.Conv2d(channel_1, channel_2, 3, padding = 1),
                       nn.ReLU(),
                       Flatten(),
                       nn.Linear(channel_2 * 32 * 32, 10)
                     )
optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9, nesterov = True)
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part(model, optimizer)

Iteration 0, loss = 2.2958
Checking accuracy on validation set
Got 95 / 1000 correct (9.50)

Iteration 100, loss = 1.8490
Checking accuracy on validation set
Got 329 / 1000 correct (32.90)

Iteration 200, loss = 1.5278
Checking accuracy on validation set
Got 403 / 1000 correct (40.30)

Iteration 300, loss = 1.5672
Checking accuracy on validation set
Got 410 / 1000 correct (41.00)

Iteration 400, loss = 1.7135
Checking accuracy on validation set
Got 449 / 1000 correct (44.90)

Iteration 500, loss = 1.5885
Checking accuracy on validation set
Got 457 / 1000 correct (45.70)

Iteration 600, loss = 1.4965
Checking accuracy on validation set
Got 495 / 1000 correct (49.50)

Iteration 700, loss = 1.5100
Checking accuracy on validation set
Got 478 / 1000 correct (47.80)

Iteration 800, loss = 1.3973
Checking accuracy on validation set
Got 535 / 1000 correct (53.50)

Iteration 900, loss = 1.0456
Checking accuracy on validation set
Got 533 / 1000 correct (53.30)

Iteration 1000, loss = 1.5473
Chec